#### Live CodeBench датасет

[LiveCodeBench](https://huggingface.co/datasets/livecodebench/test_generation) — это бенчмарк для оценки возможностей языковых моделей (LLMs) в программировании: генерации кода, самовосстановления, предсказания тестов и выполнения кода. Задачи взяты с сайтов соревновательного программирования, обеспечивая высокое качество и разнообразие сложности. Сейчас включает 442 задания из 185 задач LeetCode, цель — предсказать результат для заданного ввода.

Посмотрим на общий вид датасета

In [ ]:
import pandas as pd

def load_dataset(dataset_path = './data/live_codebench_dataset/test.parquet'):
	'''Функция считывает датасет в формате .parquet и возвращает его'''
	code_dataset = pd.read_parquet(dataset_path)
	return code_dataset

code_dataset = load_dataset()
code_dataset.head(5)

,question_title,question_content,question_id,contest_id,test_id,contest_date,starter_code,function_name,difficulty,test
0,number-of-senior-citizens,You are given a 0-indexed array of strings det...,2727,biweekly-contest-104,0,2023-05-13,"def countSeniors(self, details: List[str]) -> ...",countSeniors,easy,"[{""input"": ""[\""7868190130M7522\"", \""5303914400..."
1,number-of-senior-citizens,You are given a 0-indexed array of strings det...,2727,biweekly-contest-104,1,2023-05-13,"def countSeniors(self, details: List[str]) -> ...",countSeniors,easy,"[{""input"": ""[\""1313579440F2036\"", \""2921522980..."
2,sum-in-a-matrix,You are given a 0-indexed 2D integer array num...,2728,biweekly-contest-104,0,2023-05-13,"def matrixSum(self, nums: List[List[int]]) -> ...",matrixSum,medium,"[{""input"": ""[[7, 2, 1], [6, 4, 2], [6, 5, 3], ..."
3,sum-in-a-matrix,You are given a 0-indexed 2D integer array num...,2728,biweekly-contest-104,1,2023-05-13,"def matrixSum(self, nums: List[List[int]]) -> ...",matrixSum,medium,"[{""input"": ""[[1]]"", ""output"": ""1"", ""testtype"":..."
4,maximum-or,You are given a 0-indexed integer array nums o...,2730,biweekly-contest-104,0,2023-05-13,"def maximumOr(self, nums: List[int], k: int) -...",maximumOr,medium,"[{""input"": ""[12, 9]\n1"", ""output"": ""30"", ""test..."


Размер датасета

In [4]:
print(f"Число сэмплов: {code_dataset.shape[0]}")
print(f"Число колонок в датасете: {code_dataset.shape[1]}")

Число сэмплов: 442
Число колонок в датасете: 10


Статистики + Инфо

In [5]:
code_dataset.describe()

,test_id,contest_date
count,442.000000,442
mean,0.766968,2023-09-27 04:46:41.809954816
min,0.000000,2023-05-07 00:00:00
25%,0.000000,2023-07-16 00:00:00
50%,1.000000,2023-09-24 00:00:00
75%,1.000000,2023-12-09 00:00:00
max,3.000000,2024-03-02 00:00:00
std,0.736169,NaN


In [6]:
code_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   question_title    442 non-null    object        
 1   question_content  442 non-null    object        
 2   question_id       442 non-null    object        
 3   contest_id        442 non-null    object        
 4   test_id           442 non-null    int64         
 5   contest_date      442 non-null    datetime64[ns]
 6   starter_code      442 non-null    object        
 7   function_name     442 non-null    object        
 8   difficulty        442 non-null    object        
 9   test              442 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 34.7+ KB


Релевантные для задачи колонки:
* *question_title* - что-то может сказать про смысл задачи
* *question_content* - описание задания на естественнном языке
* *starter_code* - шаблон для написания функции типа:
	```python
	class Solution:
		def method_name(self, params) -> type:
			pass
* *function_name* - аналогично question_title
* *tes*t* - непосредственно тесты (target)


Посмотрим на эти колонки пристальнее

In [14]:
relevant_columns = [
    'question_title',
	'question_content',
	'starter_code', 
	'function_name',
	'test'
    ]

def len_starter_code(code):
    '''Функция, которая посмотрит на число символов в коде'''
    return len(code)

def process_columns(dataset = code_dataset, columns = relevant_columns):
	'''Простая функция, которая просто что-то сделает с колонками, например, unique посчитает или длину'''
	for column in columns:
		print(f"Column {column}:")
		print(f"Nunique: {dataset[column].nunique()}")
		if column == 'starter_code':
			dataset['len_code'] = dataset['starter_code'].map(lambda x: len_starter_code(x))
			print(f"Mean len: {dataset['len_code'].mean()}")
			print(f"Max len: {dataset['len_code'].max()}")
			print(f"Min len: {dataset['len_code'].min()}")
			print(f"Code with max len:")
			code_max_len = dataset[dataset['len_code'] == dataset['len_code'].max()]['starter_code'].values[0]
			print(code_max_len)
		print()

process_columns()
	

Column question_title:
Nunique: 182

Column question_content:
Nunique: 182

Column starter_code:
Nunique: 179
Mean len: 67.16289592760181
Max len: 122
Min len: 44
Code with max len:
def minimumCost(self, source: str, target: str, original: List[str], changed: List[str], cost: List[int]) -> int:
    pass

Column function_name:
Nunique: 167

Column test:
Nunique: 438



#### Резюме

Основные особенности датасета:

* Небольшой и с небольшим числом уникальных сэмплов по части кода

* Вероятно, нерелевантен для решаемой задачи, так как не содержит написанного кода и имеет упор на конкретные задания на LeetCode.

* Может быть использован для дополнительных сэмплов типа "шаблон для функции, задание для коммента -> тест к коду"
